# Program Sets + Mitiq Demo

This notebook demonstrates how to pair Amazon Braket Program Sets with Mitiq zero-noise extrapolation (ZNE). The goal is to batch together scaled circuits, evaluate their observables in a single request, and post-process the results to reduce noise.


## Prerequisites

Install dependencies with `pip install -r requirements.txt` or `poetry install`.

Once the environment is ready, run `python examples/run_program_set_demo.py` for the script version or execute the cells below.


## Workflow Outline

1. Build a reference two-qubit circuit.
2. Generate scaled copies with Mitiq for zero-noise extrapolation.
3. Package the circuits and observables into a `ProgramSet`.
4. Run the batch on the local Braket simulator and combine results.


In [1]:
from math import pi
from braket.circuits import Circuit
from braket.program_sets import ProgramSet
from braket.circuits.observables import I, X, Y, Z
import mitiq

In [2]:
circuit = Circuit().rx(0, pi / 4).ry(1, pi / 8).cnot(1, 0)
qem_circuits = mitiq.zne.construct_circuits(circuit, scale_factors=[1, 3, 5])

observable = Z(0) @ Z(1)

In [3]:
program_set = ProgramSet.product(
    circuits=qem_circuits,
    observables=[observable, I(0) @ I(1)],
)

In [4]:
from braket.devices import LocalSimulator

device = LocalSimulator()
result_1 = device.run(program_set, shots=6).result()

In [5]:
mitiq.zne.combine_results(
    [1, 3, 5],
    [result_1[i][0].expectation for i in range(3)],
    extrapolation_method=mitiq.zne.inference.LinearFactory.extrapolate,
)
[result_1[i][0].expectation for i in range(3)]


[1.0, -1.0, 1.0]

## Next Steps

- Swap `LocalSimulator` in `run_program_set` for another Braket device once your AWS credentials are configured.
- Experiment with alternative extrapolation factories from Mitiq (for example, `RichardsonFactory`).
- Add more observables to the `ProgramSet.product` call for richer measurement batches.
